In [1]:
from hhl_openai_agents_sdk.models import get_gemini
from dotenv import load_dotenv
import nest_asyncio

nest_asyncio.apply()
load_dotenv()
model = get_gemini()

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


In [4]:
from agents import Agent, function_tool


@function_tool
def book_flight(source: str, destination: str, date: str) -> str:
    confirmation_id = "ABC123"
    cost = 100.0
    print('Book Agent Called', source, destination, date)
    return f"Flight booked from {source} to {destination} on {date}, confirmation id: {confirmation_id}, total cost: ${cost}."


@function_tool
def refund_flight(confirmation_id: str) -> str:
    print('Refund Agent Called', confirmation_id)
    return f"Flight with booking ID {confirmation_id} has been refunded."


booking_agent = Agent(
    name="Booking agent",
    model=model,
    instructions="Help the customer to book flight.",
    tools=[book_flight],
)

refund_agent = Agent(
    name="Refund agent",
    model=model,
    instructions="Help the customer to cancel flight and refund the money back to customer using the refund tool.",
    tools=[refund_flight],
)

triage_agent = Agent(
    name="Triage agent",
    model=model,
    instructions=(
        "Help the user with their questions, make sure to use the handoff agent tools, don't reply to user anything other than the tool response\n"
        "\n - If they ask about flight booking, use the flight booking agent tool."
        "\n - If they ask about flight refunds, use the flight refund agent tool."
    ),

    handoffs=[booking_agent, refund_agent],
)

In [5]:
from agents import Runner

result = await Runner.run(
    triage_agent,
    "I want to book a flight from New York to San Francisco on 2023-10-15.",
)

print(result.final_output)
new_input = result.to_input_list() + [{"role": "user", "content": "I want to cancel my flight with booking ID ABC123."}]
result = await Runner.run(
    triage_agent,
    new_input,
)
print(result.final_output)

OK. I will transfer you to a booking agent.

OK. I will transfer you to a refund agent.



In [6]:
result.to_input_list()

[{'content': 'I want to book a flight from New York to San Francisco on 2023-10-15.',
  'role': 'user'},
 {'id': '__fake_id__',
  'arguments': '{}',
  'call_id': '',
  'name': 'transfer_to_booking_agent',
  'type': 'function_call'},
 {'call_id': '',
  'output': "{'assistant': 'Booking agent'}",
  'type': 'function_call_output'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    'text': 'OK. I will book a flight from New York to San Francisco on 2023-10-15 for you.\n',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'},
 {'id': '__fake_id__',
  'arguments': '{"source":"New York","date":"2023-10-15","destination":"San Francisco"}',
  'call_id': '',
  'name': 'book_flight',
  'type': 'function_call'},
 {'call_id': '',
  'output': 'Flight booked from New York to San Francisco on 2023-10-15, confirmation id: ABC123, total cost: $100.0.',
  'type': 'function_call_output'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    '

In [4]:
from agents import Runner
result = Runner.run_sync(
    triage_agent,
    "I want to cancel my flight with confirmation ID: ABC123.",
)
print(result.final_output)

OK. I will transfer you to a refund agent who can help you with that.

